In [1]:
from IPython.display import HTML as html, HTML
from IPython.display import display
import sympy as sp
import inspect

def retrieve_name(var):# Used to identify the string used to name an expression
    callers_local_vars = inspect.currentframe().f_back.f_back.f_locals.items()
    string_name = ''
    for var_name, var_val in callers_local_vars:
        if (var_val is var) and not (var_name.startswith('_')):
            string_name = var_name
    return string_name

def search_globals(var):
    string_name = ''
    for var_name in globals():
        if (globals()[var_name]==var) and not (var_name.startswith('_')):
            string_name = var_name
    return string_name
        
def dmo_auto(status=True):
    '''
    sets the status of dmo (display math operation) to True by default or False if `status=False`
    is passed to the function. If true dmo aware functions will attempt to display typeset
    expressions for the operations carried out. If False they will only display typeset expressions
    for the operations if `display_op=True` is set in the function call. See the documentation for
    each function.
    '''
    globals()['_dmo_auto_']=status
    return('Automatic display of math operation activated for `dmo aware` operations.')

def dmo(*exprs,**kwarg): #Display math operation
    '''
    Pass one of:
    *exprs:     any valid symbolic math expression or function call (e.g. 2*x, sin(2*x)).
    **kwarg:    any valid symbol equal to an expression or function call (e.g. P=n*R*T/V,
                r=sin(theta)). This must be the first item with an equal sign appearing
                in the list of arguments.
                
                additionally you can add these options:
                code=True, if you want the code version of the evaluated expression to
                    appear in the cell output. Useful if you want to copy and edit into
                    another code cell. Note if `sympy.init_printing(pretty_print=True)` is
                    set the code will still be displayed as typeset. Call `sympy.init_printing`
                    with `pretty_print=False` to get non-typeset output.
    
    In an IPython/Jupyter notebook will display a typeset version of the operation and return
    the expression to the namespace for further operations. The function tries to identify
    the actual string (name) passed for the expression and output the typeset expression 
    `name=result of the operation`. If it cannot identify the actual string passed it just
    outputs the typeset result of the operation.
    
    J. Gutow May 2020.
    '''
    code = kwarg.pop('code', None)
    expr=None
    if(len(exprs) >= 1):
        expr = exprs[0] #ignore others.
        #namestr=retrieve_name(expr)
        namestr=search_globals(expr)
        if (namestr==''):
            display(html(r'$'+sp.latex(expr)+'$'))
        else:
            display(html(r'$'+namestr+'='+sp.latex(expr)+'$'))
    else:
        key = list(kwarg)[0] #ignore all but first.
        expr = kwarg[key]
        display(html(r'$'+(key)+'='+sp.latex(expr)+'$'))
        globals()[key]=expr #inject into namespace.
    globals()['_']=expr #inject into last result
    if (code):
        return(expr)
    pass

In [2]:
# Turning off sympy auto pretty printing as that seems to be the default in latest Jupyter/Sympy.
sp.init_printing(pretty_print=False)
# Turning on auto display math operations
dmo_auto()

'Automatic display of math operation activated for `dmo aware` operations.'

In [3]:
#Integration
def integ(f, *args, **kwargs):
    display_op=kwargs.pop('display_op',None) # For getting nice display from bare function.
    result=sp.integrate(f,*args,**kwargs)
    try:
        dmo_auto = globals()['_dmo_auto_']
    except KeyError:
        dmo_auto=False
    if (display_op==False): #force overide of dmo_auto
        dmo_auto=False
    oper=None
    if (dmo_auto) or (display_op):
        #namestr=retrieve_name(f)
        namestr=search_globals(f)
        ltop1=''
        if not(namestr==''):
            oper1=sp.Integral(namestr,*args,**kwargs)
            ltop1=sp.latex(oper1)+'='
        oper=sp.Integral(f,*args,**kwargs)
        #do substitution of namestr into latex of operation?
        display(HTML('$'+ltop1+sp.latex(oper)+'='+sp.latex(result)+'$'))
    return(result)

In [4]:
sp.integrate?

In [5]:
sp.var('a b c')
dmo(sp.integrate(a*sp.exp(-b**2/c),b))

In [6]:
dmo(sp.Integral(a*sp.exp(-b**2/c),b))

In [7]:
integ(a*sp.exp(-b**2/c),b)

sqrt(pi)*a*erf(b*sqrt(1/c))/(2*sqrt(1/c))

In [8]:
q=a*b/c**2
integ(q,c)

-a*b/c

In [9]:
integ(q,c,display_op=False)

-a*b/c

In [10]:
dmo(integ(q,c,display_op=False))
dmo(integ(q,c))

In [11]:
dmo(r = integ(q,c))

In [12]:
#pass a kwarg?
nop={'display_op':False}
integ(q,c,eval(nop))

TypeError: eval() arg 1 must be a string, bytes or code object

In [13]:
#trig ?
def sin(f, **kwargs):
    def subs(*args,**kwargs):
        return(sp.sin.subs(*args,**kwargs))
        
    display_op=kwargs.pop('display_op',None) # For getting nice display from bare function.
    result=sp.sin(f,**kwargs)
    try:
        dmo_auto = globals()['_dmo_auto_']
    except KeyError:
        dmo_auto=False
    if (display_op==False): #force overide of dmo_auto
        dmo_auto=False
    oper=None
    if (dmo_auto) or (display_op):
        #namestr=retrieve_name(f)
        namestr=search_globals(f)
        ltop1=''
        if not(namestr==''):
            oper1=sp.sin(namestr,evaluate=False)
            ltop1=sp.latex(oper1)+'='
        kwargs['evaluate']=False
        oper=sp.sin(f,**kwargs)
        ltoper=''
        if not(oper==result):
            ltoper=sp.latex(oper)+'='            
        display(HTML('$'+ltop1+ltoper+sp.latex(result)+'$'))
    return(result)

In [14]:
sp.sin(1.2)

0.932039085967226

In [15]:
sin(2*a/4/b)

sin(a/(2*b))

In [16]:
sin(1.2)
dmo(h=sin(1.2))

In [17]:
dmo(h2=sin(2*a/4/b))

In [18]:
sin(q)

sin(a*b/c**2)

In [19]:
dmo(h3=sin(q))

In [20]:
n1= 1.2
sin(n1)

0.932039085967226

In [21]:
sp.var('n2')
dmo(sin(n2).subs(n2,a))

In [22]:
dmo(h3=sin(n2))

In [23]:
h3.subs({n2:1.2})

0.932039085967226

In [24]:
dmo(h3.subs(n2,1.2))

In [ ]:
sp.sin.

In [25]:
n2 = 1.2
dmo(h5=sin(n2))

In [26]:
h3.evalf(subs={n2:1.2})

sin(n2)

In [27]:
a*n2

1.2*a

In [28]:
dmo(g1=sp.sin(2*a/b)-c)

In [29]:
dmo(g1.evalf(subs={c:1.2,b:3}))

In [30]:
g1.evalf(subs={c:1.2,b:3})

-c + sin(2*a/b)

In [31]:
g1

-c + sin(2*a/b)

In [32]:
g2=sp.sin(2*a/b)-c
g2

-c + sin(2*a/b)

In [33]:
g2.subs(c,1.2)

sin(2*a/b) - 1.2

In [34]:
dmo(g2.subs(c,1.2))

In [35]:
dmo(g3=sp.sin(2*a/b)-c)

In [36]:
g3.subs(c,1.2)

sin(2*a/b) - 1.2

In [37]:
dmo(g3.subs(c,1.2))

In [38]:
dmo(g4=sp.sin(2*a/b)-c)
dmo(g4.subs(c,1.2))

In [39]:
g4

-c + sin(2*a/b)

In [40]:
g4.evalf(subs={c:1.2,a:2,b:3})

-0.228062098636687

In [41]:
search_globals(-c + sin(2*a/b))

'g4'

In [42]:
g4

-c + sin(2*a/b)

In [43]:
dmo(p=4*a/b-c)

In [44]:
search_globals(4*a/b-c)

'p'

In [5]:
sp.var('lm')
dmo(srt=sp.sqrt(lm))

In [6]:
dmo(srt.subs(lm,16))

In [7]:
dmo(srt)

In [8]:
type(sp.sin)

sympy.core.function.FunctionClass

In [34]:
import inspect
def get_ipython_globals():
    is_not_ipython_global=True
    global_dict=inspect.currentframe().f_globals
    try:
        docstr=global_dict['__doc__']
        #print(docstr)
    except KeyError:
        docstr=''
    if (docstr=='Automatically created module for IPython interactive environment'):
        is_not_ipython_global=False
    try:
        frame = inspect.currentframe().f_back
        nextframe=frame 
        while (is_not_ipython_global):
            nextframe = frame.f_back
            frame=nextframe
            try:
                global_dict=frame.f_globals
                docstr=global_dict['__doc__']
            except KeyError:
                docstr=''
            if (docstr=='Automatically created module for IPython interactive environment'):
                is_not_ipython_global=False
    except:
        pass
    return(global_dict)

In [36]:
get_ipython_globals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  "from IPython.display import HTML as html, HTML\nfrom IPython.display import display\nimport sympy as sp\nimport inspect\n\ndef retrieve_name(var):# Used to identify the string used to name an expression\n    callers_local_vars = inspect.currentframe().f_back.f_back.f_locals.items()\n    string_name = ''\n    for var_name, var_val in callers_local_vars:\n        if (var_val is var) and not (var_name.startswith('_')):\n            string_name = var_name\n    return string_name\n\ndef search_globals(var):\n    string_name = ''\n    for var_name in globals():\n        if (globals()[var_name]==var) and not (var_name.startswith('_')):\n            string_name = var_name\n    return string_name\n        \ndef dmo_auto

In [27]:
globals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  "from IPython.display import HTML as html, HTML\nfrom IPython.display import display\nimport sympy as sp\nimport inspect\n\ndef retrieve_name(var):# Used to identify the string used to name an expression\n    callers_local_vars = inspect.currentframe().f_back.f_back.f_locals.items()\n    string_name = ''\n    for var_name, var_val in callers_local_vars:\n        if (var_val is var) and not (var_name.startswith('_')):\n            string_name = var_name\n    return string_name\n\ndef search_globals(var):\n    string_name = ''\n    for var_name in globals():\n        if (globals()[var_name]==var) and not (var_name.startswith('_')):\n            string_name = var_name\n    return string_name\n        \ndef dmo_auto